"Let’s compare the different confidence assessment strategies for decoder models. Test
the Llama-2 3B model with facts from Wikipedia pages. Try the margin sampling
method, the self-consistency method, and just asking the LLM how confident it is
about the answer. Which method do you observe is a better representation of confidence
of the LLM?"

In [1]:
! pip3 install transformers datasets sentence_transformers

In [2]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd

In [4]:
model_name = "meta-llama/Llama-3.2-1B" # note to reader: make sure your token allows for write access
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [13]:
def margin_sampling(user_prompt):
    """
    Implements margin sampling by evaluating the difference in probabilities between the top two token predictions.

    Args:
        user_prompt (str): The input prompt for the language model.

    Returns:
        float: Difference in probabilities between the top two tokens.
    """
    # Tokenize the input prompt into tensor format.
    inputs = tokenizer(user_prompt, return_tensors="pt")

    # Get logits from the model with sampling enabled for diversity.
    with torch.no_grad():
        outputs = model(**inputs, temperature=0.7, do_sample=True)
        logits = outputs.logits

    # Extract logits for the last token prediction.
    last_token_logits = logits[0, -1, :]

    # Convert logits to probabilities.
    probs = torch.nn.functional.softmax(last_token_logits, dim=-1)

    # Retrieve the top 5 tokens and their probabilities.
    top_probs, top_indices = torch.topk(probs, 5)

    print("Prompt: ", user_prompt)

    # Print the top 5 tokens with their probabilities.
    for i in range(5):
        token = tokenizer.decode([top_indices[i].item()])
        prob = top_probs[i].item()
        print(f"Token: {repr(token)} | Probability: {prob:.4f}")

    # Return the margin (difference) between the top 2 token probabilities.
    return top_probs[0] - top_probs[1]


margin_sampling("The capital of France is")
print('\n')
margin_sampling("The iPhone came out in")
print('\n')
margin_sampling("Most lizards are quadrupedal, running with a strong side-to-side")
print('\n')
margin_sampling("The Franco-Prussian War or Franco-German War, often referred to in France as the War of 1870, was a conflict between the Second French Empire and the North German Confederation led by the")


Prompt:  The capital of France is
Token: ' Paris' | Probability: 0.3916
Token: ' a' | Probability: 0.0842
Token: ' the' | Probability: 0.0704
Token: ' one' | Probability: 0.0310
Token: ' also' | Probability: 0.0306


Prompt:  The iPhone came out in
Token: ' ' | Probability: 0.7889
Token: ' the' | Probability: 0.0576
Token: ' June' | Probability: 0.0330
Token: ' September' | Probability: 0.0208
Token: ' January' | Probability: 0.0084


Prompt:  Most lizards are quadrupedal, running with a strong side-to-side
Token: ' motion' | Probability: 0.2743
Token: ' g' | Probability: 0.2157
Token: ' movement' | Probability: 0.0884
Token: ' stride' | Probability: 0.0355
Token: ' body' | Probability: 0.0237


Prompt:  The Franco-Prussian War or Franco-German War, often referred to in France as the War of 1870, was a conflict between the Second French Empire and the North German Confederation led by the
Token: ' German' | Probability: 0.5831
Token: ' Kingdom' | Probability: 0.1155
Token: ' Pr' | Prob

tensor(0.4676)

In [12]:
def self_consistency(user_prompt, num_tries=5):
    """
    Implements self-consistency sampling by generating multiple responses and extracting the most probable token each time.

    Args:
        user_prompt (str): The input prompt for the language model.
        num_tries (int): Number of times to sample responses (default: 5).

    Returns:
        pd.DataFrame: A DataFrame containing the top token and its probability for each sampling iteration.
    """
    # Tokenize the input prompt for the model.
    inputs = tokenizer(user_prompt, return_tensors="pt")

    # Initialize a list to store results as (token, probability) pairs.
    result = [['token', 'probability']]

    print("Prompt: ", user_prompt)

    for i in range(num_tries):
        # Get logits from the model using sampling temperature for diversity.
        with torch.no_grad():
            outputs = model(**inputs, temperature=0.7, do_sample=True)
            logits = outputs.logits

        # Extract logits for the last token prediction.
        last_token_logits = logits[0, -1, :]

        # Convert logits to probabilities.
        probs = torch.nn.functional.softmax(last_token_logits, dim=-1)

        # Find the most probable token (argmax).
        idxmax = torch.argmax(probs)

        # Decode the token and get its probability.
        token = tokenizer.decode([idxmax.item()])
        prob = probs[idxmax].item()
        print(f"Token: {repr(token)} | Probability: {prob:.4f}")

        # Append token and its probability to results.
        result.append([token, prob])

    # Convert result list into a DataFrame for easy analysis.
    return pd.DataFrame(result[1:], columns=result[0])



self_consistency("The capital of France is")
print('\n')
self_consistency("The iPhone came out in")
print('\n')
self_consistency("Most lizards are quadrupedal, running with a strong side-to-side")
print('\n')
self_consistency('The Franco-Prussian War or Franco-German War, often referred to in France as the War of 1870, was a conflict between the Second French Empire and the North German Confederation led by the')

Prompt:  The capital of France is
Token: ' Paris' | Probability: 0.3916
Token: ' Paris' | Probability: 0.3916
Token: ' Paris' | Probability: 0.3916
Token: ' Paris' | Probability: 0.3916
Token: ' Paris' | Probability: 0.3916


Prompt:  The iPhone came out in
Token: ' ' | Probability: 0.7889
Token: ' ' | Probability: 0.7889
Token: ' ' | Probability: 0.7889
Token: ' ' | Probability: 0.7889
Token: ' ' | Probability: 0.7889


Prompt:  Most lizards are quadrupedal, running with a strong side-to-side
Token: ' motion' | Probability: 0.2743
Token: ' motion' | Probability: 0.2743
Token: ' motion' | Probability: 0.2743
Token: ' motion' | Probability: 0.2743
Token: ' motion' | Probability: 0.2743


Prompt:  The Franco-Prussian War or Franco-German War, often referred to in France as the War of 1870, was a conflict between the Second French Empire and the North German Confederation led by the
Token: ' German' | Probability: 0.5831
Token: ' German' | Probability: 0.5831
Token: ' German' | Probabilit

,token,probability
0,German,0.583111
1,German,0.583111
2,German,0.583111
3,German,0.583111
4,German,0.583111


In [20]:
def query_LLM(user_query):
    """
    Generates a response from the LLM given a user query.

    Args:
        user_query (str): Input prompt to the language model.

    Returns:
        None: Prints the decoded response from the model.
    """
    # Tokenize the input query into tensor format suitable for the model.
    inputs = tokenizer(user_query, return_tensors="pt")

    # Generate output from the model without gradient computation.
    with torch.no_grad():
        outputs = model.generate(**inputs)

    # Decode the generated token IDs into a human-readable string.
    decoded_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(decoded_text)


query_LLM("What is the capital of France and how confident are you in your answer?")
print('\n')
query_LLM("How confident are you in your answer: The iPhone came out in")
print('\n')
query_LLM("How confident are you in your answer: Most lizards are quadrupedal, running with a strong side-to-side")
print('\n')
query_LLM('How confident are you in your answer: The Franco-Prussian War or Franco-German War, often referred to in France as the War of 1870, was a conflict between the Second French Empire and the North German Confederation led by the')

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


What is the capital of France and how confident are you in your answer? Is it Lyon, Paris, Nantes, or Marseille?
France is the capital of France, and




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


How confident are you in your answer: The iPhone came out in 2007, and was the first smartphone. It was the first smartphone with a touchscreen, and




Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


How confident are you in your answer: Most lizards are quadrupedal, running with a strong side-to-side motion. The tail is usually small and the feet are long and powerful. Most lizards are quadr


How confident are you in your answer: The Franco-Prussian War or Franco-German War, often referred to in France as the War of 1870, was a conflict between the Second French Empire and the North German Confederation led by the German Empire, which resulted in the defeat of France. The war was fought from 19 July 
